<a href="https://colab.research.google.com/github/danielsuassuna04/TransferLearning-keras/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras

2024-09-18 17:10:44.061595: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 17:10:44.061803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 17:10:44.182798: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-18 17:10:44.435082: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-18 17:10:47.037669: W tensorflow/compiler/tf2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import preprocess_input


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()


batch_size = 32


def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_dataset = (train_dataset
                 .map(lambda x, y: preprocess(x, y))
                 .shuffle(buffer_size=10000)
                 .batch(batch_size)
                 .prefetch(1))
test_dataset = (test_dataset
                .map(lambda x, y: preprocess(x, y))
                .batch(batch_size)
                .prefetch(1))

2024-09-18 17:10:53.557519: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-18 17:10:53.980292: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-18 17:10:53.980530: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
base_model = keras.applications.ResNet50(weights="imagenet",include_top=False)

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
x = keras.layers.Dropout(0.5)(avg)
output = keras.layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=base_model.input,outputs=output)

In [ ]:
for layers in base_model.layers:
    layers.trainable = False

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01,weight_decay=0.01)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
early = keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    save_weights_only=True,
    verbose=1
)
history = model.fit(train_dataset,epochs=20,validation_data=test_dataset,callbacks=[early,checkpoint])

Epoch 1/20


2024-09-18 17:11:14.571560: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 9914 of 10000
2024-09-18 17:11:14.631260: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.
2024-09-18 17:11:17.408959: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-09-18 17:11:23.213622: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f26fcd783a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-18 17:11:23.213715: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro P5000, Compute Capability 6.1
2024-09-18 17:11:23.237978: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


   1/1563 [..............................] - ETA: 9:44:32 - loss: 3.4834 - accuracy: 0.1562

I0000 00:00:1726679483.424128    4400 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 [==============================] - ETA: 0s - loss: 1.9483 - accuracy: 0.7948
Epoch 1: val_loss improved from inf to 1.72153, saving model to best_model.weights.h5
1563/1563 [==============================] - 177s 99ms/step - loss: 1.9483 - accuracy: 0.7948 - val_loss: 1.7215 - val_accuracy: 0.8446
Epoch 2/20


2024-09-18 17:14:08.115641: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:3: Filling up shuffle buffer (this may take a while): 9072 of 10000


   2/1563 [..............................] - ETA: 2:09 - loss: 3.4304 - accuracy: 0.7969   

2024-09-18 17:14:09.025605: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


1562/1563 [============================>.] - ETA: 0s - loss: 2.3798 - accuracy: 0.8215
Epoch 2: val_loss improved from 1.72153 to 1.48961, saving model to best_model.weights.h5
1563/1563 [==============================] - 164s 98ms/step - loss: 2.3793 - accuracy: 0.8216 - val_loss: 1.4896 - val_accuracy: 0.8766
Epoch 3/20
1562/1563 [============================>.] - ETA: 0s - loss: 2.4620 - accuracy: 0.8266
Epoch 3: val_loss did not improve from 1.48961
1563/1563 [==============================] - 159s 97ms/step - loss: 2.4614 - accuracy: 0.8266 - val_loss: 1.6303 - val_accuracy: 0.8796
Epoch 4/20
1562/1563 [============================>.] - ETA: 0s - loss: 2.4864 - accuracy: 0.8331
Epoch 4: val_loss did not improve from 1.48961
1563/1563 [==============================] - 160s 97ms/step - loss: 2.4865 - accuracy: 0.8331 - val_loss: 1.5516 - val_accuracy: 0.8777
Epoch 5/20
1562/1563 [============================>.] - ETA: 0s - loss: 2.5546 - accuracy: 0.8308
Epoch 5: val_loss did not i

In [ ]:
model.load_weights("best_model.weights.h5")

In [ ]:
model.evaluate(test_dataset)

313/313 [==============================] - 24s 77ms/step - loss: 1.4896 - accuracy: 0.8766


[1.4896103143692017, 0.8766000270843506]